# Crear Web API para utilizar modelo entrenado con TensorFlow+Keras usando Flash y ngrok
Fuentes:

https://blog.keras.io/building-a-simple-keras-deep-learning-rest-api.html

https://deeplizard.com/learn/video/SI1hVGvbbZ4

https://curiousily.com/posts/deploy-keras-deep-learning-project-to-production-with-flask/

https://pyngrok.readthedocs.io/en/latest/integrations.html#google-colaboratory

In [1]:
#@title Accede al Drive

# Acceder al drive
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
#@title Cargar Modelo ya entrenado
import os
import joblib
import tensorflow
from tensorflow.keras.models import load_model

path_modelo = '/content/gdrive/My Drive/IA/demoModelDeployment/modelo'  #@param {type:"string"}

# cargar modelo
model = load_model(path_modelo)
print("\n* Modelo cargado de ", path_modelo, "\n")
model.summary()

# cargar scaler (si existe)
fn_scaler = path_modelo+"/scaler.joblib"
if os.path.isfile(fn_scaler):
  scaler = joblib.load(fn_scaler)
  print("\n* Scaler cargado de ", fn_scaler, "\n")
else:
  scaler = None
  print("\n* Scaler no encontrado en ", fn_scaler, "\n")




* Modelo cargado de  /content/gdrive/My Drive/IA/demoModelDeployment/modelo 

Model: "RNA"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 4)]               0         
                                                                 
 hidd_1 (Dense)              (None, 12)                60        
                                                                 
 hidd_2 (Dense)              (None, 3)                 39        
                                                                 
 output (Dense)              (None, 4)                 16        
                                                                 
Total params: 115 (460.00 Byte)
Trainable params: 115 (460.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________

* Scaler no encontrado en  /content/gdrive/My Drive/IA/demoModelDeployment/mo

In [3]:
#@title Instalar pyngrok (opcional)

usar_ngrok_web_publica = True #@param{type:"boolean"}

if usar_ngrok_web_publica:
  # Instalar pyngrok
  !pip install pyngrok
  print("")
else:
  print("- no se usa ngrok.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.8/731.8 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.1-py3-none-any.whl size=21122 sha256=219dacce1b9b02c43e2174a8894ff81f8bc5dc68283b2ca1a8952cc20fef8bb4
  Stored in directory: /root/.cache/pip/wheels/3b/32/0e/27789b6fde02bf2b320d6f1a0fd9e1354b257c5f75eefc29bc
Successfully built pyngrok



In [4]:
#@title Preparar la conexión con ngrok para hacer sitio público (opcional)

ngrok_auth_token = "" #@param {type:"string"}

if usar_ngrok_web_publica:
  import getpass
  from pyngrok import ngrok, conf

  # determina el authentication token de ngrok (
  if (ngrok_auth_token == ""):
    print("Ingrese el authtoken indicada en https://dashboard.ngrok.com/get-started/your-authtoken luego de registrarse")
    conf.get_default().auth_token = getpass.getpass()
  else:
    conf.get_default().auth_token = ngrok_auth_token
  print("")
  # Open a TCP ngrok tunnel to the SSH server
  connection_string = ngrok.connect("22", "tcp").public_url
  print("")
  ssh_url, port = connection_string.strip("tcp://").split(":")
  print("")
  print(f" * se crea ngrok tunnel, accediendo con `ssh root@{ssh_url} -p{port}`")
else:
  print("- no se usa ngrok.")


Ingrese el authtoken indicada en https://dashboard.ngrok.com/get-started/your-authtoken luego de registrarse
··········





 * se crea ngrok tunnel, accediendo con `ssh root@2.tcp.ngrok.io -p10792`


In [5]:
#@title Habilitar Web API con Flash

import threading
import pandas as pd
from flask import Flask, jsonify, request, current_app
import numpy as np
import logging
import uuid
import os

# crea la application
app = Flask(__name__)

# define parámetros
local_port = "5000" #@param {type:"string"}
nombre_servicio = "/iris" #@param {type:"string"}

# define url local del servicio
urlLocal = "http://127.0.0.1:{}".format(local_port)

print("")
if usar_ngrok_web_publica:
  # Open a ngrok tunnel to the HTTP server

  public_url = ngrok.connect(local_port).public_url
  print(" * ngrok tunnel definido: \"{}\" <-> \"{}\"".format(public_url, urlLocal))

  # Update any base URLs to use the public ngrok URL
  app.config["BASE_URL"] = public_url
  # ... Update inbound traffic via APIs to use the public-facing ngrok URL
  ngrok_public_Web_API = public_url + nombre_servicio
else:
  ngrok_public_Web_API = None
  print("- no se usa ngrok.")

# configura logging en archivo
logFileName = "/content/modelWebAPI.log"
logger = logging.getLogger(__name__)
file_handel = logging.FileHandler(logFileName)
file_handel.setFormatter(logging.Formatter('%(asctime)s %(levelname)s - %(message)s'))
file_handel.setLevel(logging.DEBUG)
logger.addHandler(file_handel)

def mostrarLog():
  stream = os.popen("cat '"+ logFileName +"'")
  output = stream.read()
  print(output)

# define el servicio para clasificar flores IRIS
@app.route(nombre_servicio, methods=["POST"])
def model_web_api():
  try:
    current_app.logger.debug("+ conexion recibida: " + str(uuid.uuid4()))
    ##logger.info('Mostrando los posts del blog')
    # recibe datos de conexion
    data = request.json
    current_app.logger.debug("- datos recibidos: " + str(data))
    # formatea datos como números
    valsInput = np.array( pd.DataFrame(data, index=[0]), dtype=np.float32)
    current_app.logger.debug("- datos convertidos: " + str(valsInput))
    # si está definido el scaler normaliza los datos
    if scaler is not None:
      valsInput = scaler.transform(valsInput)
      current_app.logger.debug("- datos normalizados: " + str(valsInput))
    # ejecuta el modelo con los datos convertidos/normalizados
    resModel = model.predict(valsInput, verbose=0)
    current_app.logger.debug("- resultado del modelo: " + str(resModel))
    # se genera un resultado
    if (len(resModel) == 1):
        # identifica tipo de alida
        if (len(resModel[0]) == 1):
            # como tiene una salida se asume salida lineal (solo la redondea)
            claseID = round(resModel[0])
        else:
            # como tiene validas salidas se asume salida softmax (toma la de mayor puntaje)
            claseID = int( np.argmax(resModel[0], axis=0) )
        current_app.logger.debug("* clase asignada por modelo: " + str(claseID))
        return jsonify({"clase": str(claseID)})
    else:
        # no se genera un único resultado (error)
        current_app.logger.debug("- resultado del modelo distino de 1 elemento: " + str(resModel))
        return jsonify({"clase": "ERROR"})
  except Exception as error:
     current_app.logger.debug("!! error detectado: " + str(error))

# muestra nombre de APIs
nombre_WebAPI = "{}{}".format(urlLocal, nombre_servicio)
print("")
print(" > Web API local establecida en: ",  nombre_WebAPI)
if usar_ngrok_web_publica and (ngrok_public_Web_API is not None):
  print(" > ngrok public Web API establecida en: ",  ngrok_public_Web_API)
print("")

# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False, "port" : local_port, "debug": True}).start()



 * ngrok tunnel definido: "https://b258-34-145-200-19.ngrok-free.app" <-> "http://127.0.0.1:5000"

 > Web API local establecida en:  http://127.0.0.1:5000/iris
 > ngrok public Web API establecida en:  https://b258-34-145-200-19.ngrok-free.app/iris



In [6]:
#@title Probar Web API local con un ejemplo

import os
import json

LargoSepalo = 5.5 #@param{type:"number"}
AnchoSepalo = 2.6 #@param{type:"number"}
LargoPetalo = 4.4 #@param{type:"number"}
AnchoPetalo = 1.2 #@param{type:"number"}

dictValues = {
    "LargoSepalo" : str(LargoSepalo),
    "AnchoSepalo" : str(AnchoSepalo),
    "LargoPetalo" : str(LargoPetalo),
    "AnchoPetalo" : str(AnchoPetalo)
    }

# función auxiliar para probar API
def ejecutarModelWebAPI(dictValues, URL_API, mostrarRes=True):
  # ejecuta la web API usando curl
  cmdStr = "curl -d '" + str(dictValues).replace("'", '"') + "' -H \"Content-Type: application/json\" -X POST " + URL_API
  if mostrarRes:
    print("\n", cmdStr, "\n")
  # ejecuta y muestra resultados
  stream = os.popen(cmdStr)
  output = stream.read()
  if mostrarRes:
    print("\n --> ", output)
  # devuelve la clase
  return json.loads(output)

# ejecuta API
ejecutarModelWebAPI(dictValues, nombre_WebAPI)
print("")


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
DEBUG:__main__:+ conexion recibida: b78cd6ef-cc56-4bfa-9bde-cf9fc1e49cef
DEBUG:__main__:- datos recibidos: {'LargoSepalo': '5.5', 'AnchoSepalo': '2.6', 'LargoPetalo': '4.4', 'AnchoPetalo': '1.2'}
DEBUG:__main__:- datos convertidos: [[5.5 2.6 4.4 1.2]]



 curl -d '{"LargoSepalo": "5.5", "AnchoSepalo": "2.6", "LargoPetalo": "4.4", "AnchoPetalo": "1.2"}' -H "Content-Type: application/json" -X POST http://127.0.0.1:5000/iris 



DEBUG:__main__:- resultado del modelo: [[4.0367772e-04 4.8701672e-04 9.9134445e-01 7.7649094e-03]]
DEBUG:__main__:* clase asignada por modelo: 2
INFO:werkzeug:127.0.0.1 - - [29/Nov/2023 14:59:09] "POST /iris HTTP/1.1" 200 -



 -->  {
  "clase": "2"
}




In [7]:
#@title Probar Web API local con ejemplos de un CSV que tiene los datos mostrados en <construir-RNA-MLP-IRIS.ipynb>
from google.colab import files

def cargarArchivo(fn):

  # lo carga en lista
  uploadedData = []
  with open(fn, 'r', encoding='utf-8') as f:
    contents = f.readlines()
  # separa en lineas
  uploadedData.extend( ("\n".join(contents)).split("\n") )

  print('-- Archivo "{name}" con largo {length} bytes cargado'.format(
    name=fn, length=len(uploaded[fn])))

  return uploadedData

# sube archivo
uploaded = files.upload()
# procesa los datos
uploadedData = []
for fn in uploaded.keys():
    uploadedData.extend( cargarArchivo(fn))

print("\n")
# procesa los datos cargados
classReal = []
classPreds = []
for data in uploadedData:
  if data!="":
    arAux = data.split(";")
    dictValues = {
    "LargoSepalo" : arAux[0],
    "AnchoSepalo" : arAux[1],
    "LargoPetalo" : arAux[2],
    "AnchoPetalo" : arAux[3]
    }
    classReal.append( arAux[4] )
    claseModelo = ejecutarModelWebAPI(dictValues, nombre_WebAPI, mostrarRes=False)
    classPreds.append( claseModelo["clase"] )
print("\n")


from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd

# muestra reporte de clasificación
print("\n Reporte de Clasificación: ")
print(classification_report(classReal, classPreds))

# muestra matriz de confusion
print('\nMatriz de Confusión ( real / modelo ): ')
cm = confusion_matrix(classReal, classPreds)
cmtx = pd.DataFrame(
    cm
  )
# agrega para poder mostrar la matrix de confusión completa
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
cmtx.sort_index(axis=0, inplace=True)
cmtx.sort_index(axis=1, inplace=True)
print(cmtx)
print("\n")


DEBUG:__main__:+ conexion recibida: e83df82f-3e2b-4f01-beaa-dfba6a41c23b
DEBUG:__main__:- datos recibidos: {'LargoSepalo': '5.5', 'AnchoSepalo': '2.6', 'LargoPetalo': '4.4', 'AnchoPetalo': '1.2'}
DEBUG:__main__:- datos convertidos: [[5.5 2.6 4.4 1.2]]
DEBUG:__main__:- resultado del modelo: [[4.0367772e-04 4.8701672e-04 9.9134445e-01 7.7649094e-03]]
DEBUG:__main__:* clase asignada por modelo: 2
INFO:werkzeug:127.0.0.1 - - [29/Nov/2023 14:59:31] "POST /iris HTTP/1.1" 200 -
DEBUG:__main__:+ conexion recibida: 40f423db-ec15-4ee5-b03e-9fac8cd0dad6
DEBUG:__main__:- datos recibidos: {'LargoSepalo': '5.2', 'AnchoSepalo': '3.5', 'LargoPetalo': '1.5', 'AnchoPetalo': '0.2'}
DEBUG:__main__:- datos convertidos: [[5.2 3.5 1.5 0.2]]
DEBUG:__main__:- resultado del modelo: [[1.1355409e-11 9.9990022e-01 9.9812802e-05 5.5535888e-19]]
DEBUG:__main__:* clase asignada por modelo: 1
INFO:werkzeug:127.0.0.1 - - [29/Nov/2023 14:59:31] "POST /iris HTTP/1.1" 200 -
DEBUG:__main__:+ conexion recibida: df78ff40-d56

Saving datosPrueba.csv to datosPrueba.csv
-- Archivo "datosPrueba.csv" con largo 722 bytes cargado




DEBUG:__main__:- resultado del modelo: [[3.0947092e-04 2.2292649e-04 9.8907954e-01 1.0388126e-02]]
DEBUG:__main__:* clase asignada por modelo: 2
INFO:werkzeug:127.0.0.1 - - [29/Nov/2023 14:59:31] "POST /iris HTTP/1.1" 200 -
DEBUG:__main__:+ conexion recibida: 5461168c-462c-4f9a-8e89-728f42ceb7be
DEBUG:__main__:- datos recibidos: {'LargoSepalo': '6.3', 'AnchoSepalo': '2.3', 'LargoPetalo': '4.4', 'AnchoPetalo': '1.3'}
DEBUG:__main__:- datos convertidos: [[6.3 2.3 4.4 1.3]]
DEBUG:__main__:- resultado del modelo: [[2.5680271e-04 2.2035472e-04 9.9565685e-01 3.8660683e-03]]
DEBUG:__main__:* clase asignada por modelo: 2
INFO:werkzeug:127.0.0.1 - - [29/Nov/2023 14:59:31] "POST /iris HTTP/1.1" 200 -
DEBUG:__main__:+ conexion recibida: c2929379-e43d-4830-ba29-86a9690b1083
DEBUG:__main__:- datos recibidos: {'LargoSepalo': '7.0', 'AnchoSepalo': '3.2', 'LargoPetalo': '4.7', 'AnchoPetalo': '1.4'}
DEBUG:__main__:- datos convertidos: [[7.  3.2 4.7 1.4]]
DEBUG:__main__:- resultado del modelo: [[5.11755




 Reporte de Clasificación: 
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        13
           2       0.87      1.00      0.93        13
           3       1.00      0.83      0.91        12

    accuracy                           0.95        38
   macro avg       0.96      0.94      0.95        38
weighted avg       0.95      0.95      0.95        38


Matriz de Confusión ( real / modelo ): 
    0   1   2
0  13   0   0
1   0  13   0
2   0   2  10




In [ ]:
#@title Mostrar Archivo de Log
print("")
mostrarLog()
print("")